In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib
import IPython.display as ipd
plt.rcParams['figure.figsize'] = (17, 5)
import pandas as pd
import os
import re
import json
import codecs
from scipy import sparse, stats, spatial
from sklearn.cluster import DBSCAN, KMeans
from sklearn.mixture import GaussianMixture
import scipy.sparse.linalg
import folium
%matplotlib inline

In [ ]:
def load_party_data(path):
    data=pd.read_excel(path)
    data.drop([data.columns[0],data.columns[2],data.columns[3],data.columns[4]],1,inplace=True)
    data.drop([0,1],0,inplace=True)
    data.columns=['commune','party','percentage']
    data=data.ffill()
    data=data.groupby(['commune','party']).sum().unstack('party')
    data2=data.reset_index()['percentage']
    data2['commune']=data.reset_index()['commune']
    return data2

In [ ]:
#load data
data=load_party_data('data/partis_12prem_vote_2015.xlsx')
data2=load_party_data('data/partis_12der_vote_2015.xlsx')
#take only commune
data=data[data['commune'].str.startswith('......')]
data2=data2[data2['commune'].str.startswith('......')]
data['commune']=data['commune'].str[7:]
data2['commune']=data2['commune'].str[7:]
#merge datasets
data=pd.merge(data,data2,on='commune')
#replace non-available parties with 0
data.loc[:, data.columns != 'commune']=data.loc[:, data.columns != 'commune'].replace('...','0')
#remove data from comming from correspondancies
data=data[(data['commune'].str[:2]==(data['commune'].str.upper()).str[:2]) & (data['commune'].str[2]=='-') ==False]
data.loc[:, data.columns != 'commune']=data.loc[:, data.columns != 'commune']
data=data.set_index('commune')
data.index=data.index.str.replace(re.escape(' (Urne commune)'),'')
data=data.astype(float)
data=data.sort_index()

In [ ]:
data_tot=data.as_matrix()
data_tot
data.index.to_series()[data_tot.sum(1)==0]

commune_with_no_vote=(data_tot.sum(1)==0)

data_tot[commune_with_no_vote==False]=np.divide(data_tot[commune_with_no_vote==False],data_tot[commune_with_no_vote==False].sum(1)[:,None])

data_tab=data_tot[commune_with_no_vote==False]


In [ ]:
general=pd.read_excel('data/general_2015.xlsx')
general=general[['Unnamed: 1','Unnamed: 9']]
general.drop([0,1],0,inplace=True)
general.columns=['commune','voters']
#general=general[general['commune'].str.startswith('......')]
general=general[general['commune'].str.startswith('......',na=False)]
general['commune']=general['commune'].str[7:]
general=general[(general['commune'].str[:2]==(general['commune'].str.upper()).str[:2]) & (general['commune'].str[2]=='-') ==False]
general['voters']=general['voters'].replace('...','0')
general['voters']=general['voters'].astype(int)
general=general.set_index('commune')
general=general.sort_index()

In [ ]:
pop_tot=np.squeeze(general.as_matrix())
pop_tab=pop_tot[commune_with_no_vote==False]
#plt.hist(pop_tab,bins=50)

In [ ]:
#We defined here a list of merged communes between 2013 and 2015, with the resulting commune in the first position


fusion=[['Valbirse','Malleray','Bévilard','Pontenet'],
        ['Terre di Pedemonte','Cavigliano','Tegna','Verscio'],
        ['Val-de-Charmey','Charmey','Cerniat (FR)'],
        ['Sauge','Frinvillier','Plagne','Vauffelin'],
        ['Buchegg','Aetigkofen','Aetingen','Bibern (SO)','Brügglen','Gossliwil','Hessigkofen','Küttigkofen','Kyburg-Buchegg','Mühledorf (SO)','Tscheppach'],
        ['Domleschg','Almens','Paspels','Pratval','Rodels','Tomils'],
        ['Petit-Val','Châtelat','Monible','Sornetan','Souboz'],
        ['Ilanz/Glion','Castrisch','Ilanz','Ladir','Luven','Pitasch','Riein','Ruschein','Schnaus','Sevgein','Duvin','Pigniu','Rueun','Siat'],
        ['Péry-La Heutte','Péry','La Heutte'],
        ['Calanca','Arvigo','Braggio','Cauco','Selma'],
        ['Bettmeralp','Betten','Martisberg'],
        ['Arzier-Le Muids','Arzier'],
        ['Schinznach','Schinznach-Dorf','Oberflachs'],
        ['Albula/Alvra','Alvaschein','Mon','Stierva','Tiefencastel','Alvaneu','Brienz/Brinzauls','Surava'],
        ['Bussigny','Bussigny-près-Lausanne'],
        ['Stocken-Höfen','Niederstocken','Oberstocken','Höfen'],
        ['Plateau de Diesse','Diesse','Lamboing','Prêles'],
        ['Mendrisio','Besazio','Ligornetto','Meride'],
        ['Lugano','Bogno','Cadro','Carona','Certara','Cimadera','Sonvico','Valcolla'],
        ['Bauma','Sternenberg'],
        ['Scuol','Guarda','Ardez','Tarasp','Ftan','Sent'],
        ['Jegenstorf','Scheunen','Münchringen'],
        ['Fraubrunnen','Büren zum Hof','Etzelkofen','Grafenried','Limpach','Mülchi','Schalunen','Zauggenried'],
        ['Murten','Staatswald Galm'],
        ['Grafschaft','Kommunanz Reckingen-Gluringen/Grafschaft'],
        ['Cadenazzo','Comunanza Cadenazzo/Monteceneri'],
        ['Wiesendangen','Bertschikon'],
        ['Innertkirchen','Gadmen'],
        ['Endingen','Unterendingen'],
        ['Uttigen','Kienersrüti'],
        ['Bremgarten (AG)','Bremgarten','Hermetschwil-Staffeln'],
        ['Zernez','Lavin','Susch'],
        ['Oberdiessbach','Bleiken bei Oberdiessbach'],
        ['Vals','St. Martin']
]

In [ ]:
#since the topojson for visualisation dates back to 2013, some old municipipalities in the topojson now unified are
#missing in the dataset. Since it's complicated to find a topojson up to date, we will duplicate the data from the 
#unified municipalities to the old ones.

#list of the old munipalities fron the topjson
with open('data/gemeinden.topo.json') as f:
    muni=json.load(f)
    
s= pd.DataFrame(muni['objects']['gemeinden']['geometries'])
properties=s['properties'].values
municipalities_topo = pd.DataFrame(list(properties))['GMDNAME']
#municipalities_topo

#list of the new municipalities from dataset
#municipalities_data=data['commune']
municipalities_data = pd.Series(data.index.values)
municipalities_data.head()

#diff of both: municipalities in top no it dataset
diff_ind=~municipalities_topo.isin(municipalities_data) 
municipalities_diff= municipalities_topo[diff_ind]

#for all diff munip create new row in data:
#diff_df=pd.DataFrame(columns = data.columns, index= municipalities_diff)
for old_munip in municipalities_diff:
    #find corresponding new munip
    for i in fusion:
        for j in i:
            if old_munip == j:
                new_munip=i[0]
                #copy row from data with corresponding to new munip
                new_row=data.ix[new_munip]
                new_row = pd.DataFrame(new_row.rename(old_munip)).T
                data=data.append(new_row)


In [ ]:
w = data.sum(axis=0)/data.sum(axis=0).max()
w.values

In [ ]:
#keep only main parties
data.drop([col for col, val in data.sum().iteritems() if val == 0], axis=1, inplace=True)
data.shape

In [ ]:
general

Start of spectral clustering

In [ ]:
features = pd.DataFrame(data)

In [ ]:
#distances = spatial.distance.squareform(spatial.distance.pdist(features,'wminkowski', p=1., w=w.values))
distances = spatial.distance.squareform(spatial.distance.pdist(features,'minkowski', p=1.))
#distances = spatial.distance.squareform(spatial.distance.pdist(features,'canberra'))
#distances = spatial.distance.squareform(spatial.distance.pdist(features,'chebyshev'))
#distances = spatial.distance.squareform(spatial.distance.pdist(features,'sqeuclidean'))

In [ ]:
plt.spy(distances)

In [ ]:
print('{} distances equal exactly zero.'.format(np.sum(distances == 0)))

In [ ]:
kernel_width = distances.mean()
weights = np.exp(np.divide(-np.square(distances),kernel_width**2))
np.fill_diagonal(weights,0)
weights

In [ ]:
def plot(weights, axes):
    axes[0].spy(weights)
    axes[1].hist(weights[weights > 0].reshape(-1), bins=50);

In [ ]:
fix, axes = plt.subplots(2, 2, figsize=(17, 8))
plot(weights, axes[:, 0])

NEIGHBORS = 350

for i in range(weights.shape[0]):
    idx = weights[i,:].argsort()[:-NEIGHBORS]
    weights[i,idx] = 0
    weights[idx,i] = 0

plot(weights, axes[:, 1])

In [ ]:
degrees = np.sum(weights,axis=0)
plt.hist(degrees, bins=50);

In [ ]:
laplacian = np.diag(degrees**-0.5) @ (np.diag(degrees) - weights) @ np.diag(degrees**-0.5)
plt.spy(laplacian);

In [ ]:
laplacian = sparse.csr_matrix(laplacian)

In [ ]:
n_edges = np.sum(np.ceil(weights))/2
n_edges.astype(int)

In [ ]:
eigenvalues, eigenvectors = sparse.linalg.eigsh(A=laplacian,k=10,which='SM')

In [ ]:
plt.plot(eigenvalues, '.-', markersize=15);

In [ ]:
x = eigenvectors[:,1]
y = eigenvectors[:,2]
labels = np.sign(x)

plt.scatter(x, y, c=labels, cmap='RdBu', alpha=0.5);

In [ ]:
fix, axes = plt.subplots(5, 5, figsize=(17, 8))
for i in range(1,6):
    for j in range(1,6):
        x = eigenvectors[:,i]
        y = eigenvectors[:,j]
        labels = np.sign(x)
        axes[i-1,j-1].scatter(x, y, c=labels, cmap='RdBu', alpha=0.5)


In [ ]:
class_1 = (eigenvectors[:,2] >= 0).astype(int)
class_2 = (eigenvectors[:,3] >= 0).astype(int)
class_3 = (eigenvectors[:,4] >= 0).astype(int)
class_1

In [ ]:
labels = class_1 * 2**2 + class_2 * 2 + class_3

In [ ]:
series = pd.Series(labels, index=data.index.values)

Choropleth display :

In [ ]:
json_data_gemeinden = json.load(codecs.open('data/gemeinden.topo.json', 'r', 'utf-8-sig'))
json_data_kantone = json.load(codecs.open('data/kantone.topo.json', 'r', 'utf-8-sig'))

In [ ]:
cmap = matplotlib.cm.get_cmap('Spectral')
color_map = cmap(np.arange(0,1,1/26))


In [ ]:
def color(class_label, color_map): 
    return  {
    'fillOpacity': 0.5,
    'weight': 0.5,
    'fillColor': '#%02x%02x%02x' % tuple((256 * color_map[class_label,:3]).astype(int)),
    'color': '#%02x%02x%02x' % tuple((256 * color_map[class_label,:3]).astype(int))
     }   

In [ ]:
def style_function(data):    
    if data['properties']['GMDNAME'] in series.index:
        class_label = series[series.index == data['properties']['GMDNAME']][0]
        return color(class_label, color_map)
    
    else:
        return  {
        'fillOpacity': 0.5,
        'weight': 0.5,
        'fillColor': '#%02x%02x%02x' % tuple((256 * color_map[0,:3]).astype(int)),
        'color': '#%02x%02x%02x' % tuple((256 * color_map[0,:3]).astype(int))
         } 

In [ ]:
def map_cantons(series, json_data_gemeinden, json_data_kantone):
    center_coord = [46.8011111,8.2266667]
    cantons_map = folium.Map(location=center_coord,
                tiles='cartodbpositron',           
                zoom_start=7.5)

    scale = list(np.linspace(0.,series.max(),6))

    """
    cantons_map.choropleth(geo_data=json_data_gemeinden, topojson='objects.gemeinden', 
        data=series,
        key_on='feature.properties.GMDNAME',
        threshold_scale=scale,
        fill_color='YlOrRd', fill_opacity=0.6, line_opacity=0.3,
        highlight = True)
     
     """
    folium.TopoJson(json_data_gemeinden,'objects.gemeinden',name='gemeiden',
                    style_function=style_function).add_to(cantons_map)

    folium.TopoJson(json_data_kantone,'objects.kantone',name='cantons',style_function=lambda feature: {
            'color': 'blue',
            'fillOpacity':0.0,
            'weight': 2}).add_to(cantons_map)
    
    return cantons_map

In [ ]:
m_swiss = map_cantons(series, json_data_gemeinden, json_data_kantone)

In [ ]:
m_swiss

In [ ]:
#m_swiss.save('switzerland_spectral_clustering.html')

In [ ]:
# load geojson data for coordinates
geo_json_data = json.load(open('data/map.geojson'))

Explore geojson data : 

In [ ]:
# features -> number of municipality -> geometry -> coordinates -> coordinates of first point of arc -> x or y
# ! : make sure to get rid of extra dimensions
# example : display x coordinate of first point of arc of first municipality
#geo_json_data['features'][0]['geometry']['coordinates']
#geo_json_data['features'][653]['geometry']['coordinates'][1][0]
#np.squeeze(np.array(geo_json_data['features'][0]['geometry']['coordinates']))

In [ ]:
def rescale(features):
    # given n_sample x n_features
    min_vals = features.min(axis=0)
    max_vals = features.max(axis=0)
    
    return (features - min_vals) / (max_vals - min_vals)

In [ ]:
N = 2324
coordinates = np.empty((0, 2))
for i in range(N): 
    # calculate position as mean of point of arc
    position = np.reshape(np.array(geo_json_data['features'][i]['geometry']['coordinates'][0]),(-1,2)).mean(axis=0)

    # append to coordinate array
    coordinates = np.append(coordinates, [position], axis=0)
    
coordinates = 0.1 * rescale(coordinates)
coordinates

In [ ]:
political_features = eigenvectors[:,1:26]
#mean = political_features.mean(axis=0)
#std = political_features.std(axis=0)
#political_features = (political_features-mean)/std
#political_features = rescale(political_features)
political_features = np.sign(eigenvectors)

In [ ]:
#cluster_features = np.concatenate((political_features,coordinates),axis=1)
cluster_features = political_features

In [ ]:
#clustering_model = DBSCAN(eps=0.15, min_samples=30, metric='cityblock', p=2)
n_cantons = 26
clustering_model = KMeans(n_clusters=n_cantons, init='k-means++', 
                          n_init=20, max_iter=500, tol=0.0001)

gaussian_mixture = GaussianMixture(n_components=6, covariance_type='full', 
                                   tol=0.001, reg_covar=1e-06, max_iter=100)

In [ ]:
classes = clustering_model.fit_predict(cluster_features)
#gaussian_mixture.fit(cluster_features)
#classes = gaussian_mixture.predict(cluster_features)
classes

In [ ]:
plt.hist(classes)

### Voting misrepresentation : 
- set of voters V, i in V 
- set of Candidates C, c in C
- w winner of district
- set of districts D : D_1 ... D_z

MR(V,D,F) = max_c score_f(c,V) / score_f(w,V) --> sum MR(municipality versus canton)
instead : 
    for each canton
        sum dist(repr municipality, repr canton)

In [ ]:
idx = np.where(classes == 1)
general.iloc[idx].values.sum()

In [ ]:
def misrepresentation_score(Data,General,Classes):

    MR = []
    n_cantons = Classes.max()-1
    swiss_voters = General.values.sum()
    N_seats = 0

    # for each canton
    for canton in range(n_cantons):
        # get municipalities
        idx = np.where(Classes == canton)
        n_municipalities = len(idx)
        canton_data = Data.iloc[idx]
        canton_general = General.iloc[idx]

        # aggregate results for entire canton
        results_canton = canton_data.mean(axis=0)
        n_voters = canton_general.values.sum()
        
        # for 200 seats in total in parlement
        n_seats = math.ceil(155 * n_voters/swiss_voters)
        N_seats += n_seats
        
        #print(results_canton)
        results_canton = round(n_seats * results_canton) / (n_seats)
        #print(results_canton)
        
        # calculate average of euclidean error (misrepresentation)
        MR.append(np.mean(np.sqrt(((canton_data-results_canton)**2).sum(axis=1))))
    
    #print(N_seats)
    return np.mean(MR)

In [ ]:
misrepresentation_score(data,general,classes)

In [ ]:
# Study effect of number of clusters
cluster_range = 31

scores = []

for n_cantons in range(3,cluster_range):
    print('number of cantons : ',n_cantons)
    clustering_model = KMeans(n_clusters=n_cantons, init='k-means++', 
                          n_init=100, max_iter=500, tol=0.0001)
    classes = clustering_model.fit_predict(cluster_features)
    scores.append(misrepresentation_score(data,general,classes))
    

In [ ]:
plt.plot(scores)

In [ ]:
#municipality_labels = (classes+1)%6
municipality_labels = classes

In [ ]:
municipality_labels

In [ ]:
series = pd.Series(municipality_labels, index=data.index.values)

In [ ]:
m_swiss = map_cantons(series, json_data_gemeinden, json_data_kantone)

In [ ]:
m_swiss